# Dynamics of a 2-DoF Serial Manipulator


Import libraries

In [1]:
import sympy as p
import numpy as np
import sys

# SYMBOLIC.py is in parent directory
sys.path.append("..")
from SYMBOLIC import *

## 1. Kinematics with PoE


### 1.1 Definitions

Define time $t$ and angles.


In [2]:
t = p.Symbol("t")

# Differentialable Variables
theta_1_t = p.Function("theta_1")(t)
theta_2_t = p.Function("theta_2")(t)

# Undifferentialable Variables
theta_1, theta_2 = p.symbols("theta_1 theta_2")

# Parameters
m1, m2, l1, l2, lc1, lc2, Jc1, Jc2, g = p.symbols(
    "m1 m2 l1 l2 lc1 lc2 Jc1 Jc2 g")

Define substitution functions from differentialable variables to undifferentialable variables.

In [3]:
def ThetaT2Theta(Expression):
    return Expression.subs(theta_1_t, theta_1).subs(theta_2_t, theta_2)


def Theta2ThetaT(Expression):
    return Expression.subs(theta_1, theta_1_t).subs(theta_2, theta_2_t)

Define geometric parameters. We apply **Modified D-H** notation. Noting that $\square_{i}$ refers to $\square$ of joint $i$, $\square_{ci}$ refers to $\square$ of rod $i$.


In [4]:
# Joint DH Parameters
x0, x1, x2 = p.symbols("x_0 x_1 x_2")
y0, y1, y2 = p.symbols("y_0 y_1 y_2")
z0, z1, z2 = p.symbols("z_0 z_1 z_2")

# Link CoM Parameters
xc1, xc2 = p.symbols("x_c1 x_c2")
yc1, yc2 = p.symbols("y_c1 y_c2")
zc1, zc2 = p.symbols("z_c1 z_c2")

From the configuration, we got

In [5]:
# Joint Parameters
x0, y0, z0 = 0, 0, 0
x1, y1, z1 = 0, 0, 0
x2, y2, z2 = l1, 0, 0

# CoM Parameters
xc1, yc1, zc1 = lc1, 0, 0
xc2, yc2, zc2 = lc2, 0, 0

# RPY Angles
RPY_0_1 = [0, 0, 0]
RPY_1_2 = [0, 0, 0]
RPY_1_c1 = [0, 0, 0]
RPY_2_c2 = [0, 0, 0]

### 1.2 Forward Kinematics


Transformation matrixes at $\vec{0}$. `T_i_j_0` refers to $T_{i}^{j} (\vec{0}) $.


In [6]:
T_1_0_0 = Translation_4x4(p.Matrix([x1, y1, z1])) @ Rotation_RPY_4x4(RPY_0_1)
T_2_1_0 = Translation_4x4(p.Matrix([x2, y2, z2])) @ Rotation_RPY_4x4(RPY_1_2)
T_c1_1_0 = Translation_4x4(
    p.Matrix([xc1, yc1, zc1])) @ Rotation_RPY_4x4(RPY_1_c1)
T_c2_2_0 = Translation_4x4(
    p.Matrix([xc2, yc2, zc2])) @ Rotation_RPY_4x4(RPY_2_c2)

Joint positions at $\vec{0}$.

In [7]:
def Position(Transformation_i_j, P_i=p.Matrix([0, 0, 0, 1])):
    return (Transformation_i_j @ P_i)[:3, :]


T_1_0_0 = T_1_0_0
T_2_0_0 = T_1_0_0 @ T_2_1_0
T_c1_0_0 = T_1_0_0 @ T_c1_1_0
T_c2_0_0 = T_2_0_0 @ T_c2_2_0

r_1_0 = Position(T_1_0_0)
r_2_0 = Position(T_2_0_0)
r_c1_0 = Position(T_c1_0_0)
r_c2_0 = Position(T_c2_0_0)

`SymPy` Code Generation

In [8]:
p.printing.pycode(r_c2_0)

'ImmutableDenseMatrix([[l1 + lc2], [0], [0]])'

Joint poses at $\vec{0}$.

In [9]:
omega_1_0 = p.Matrix([0, 0, 1]).reshape(3, 1)
omega_2_0 = Rotation_RPY_4x4(RPY_1_2)[:3, :3] @ omega_1_0

Joint unitary twists at $\vec{0}$.

In [10]:
xi_1_0 = Joint2Twist(omega_1_0, r_1_0)
xi_2_0 = Joint2Twist(omega_2_0, r_2_0)

Transformation matrixes at $\vec{\theta}$ computed with PoE.


In [11]:
# Transformation Matrixes
T_1_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ T_1_0_0
T_2_0 = MatrixExp_4x4(
    xi_1_0, theta_1_t) @ MatrixExp_4x4(xi_2_0, theta_2_t) @ T_2_0_0
T_c1_0 = MatrixExp_4x4(xi_1_0, theta_1_t) @ T_c1_0_0
T_c2_0 = MatrixExp_4x4(
    xi_1_0, theta_1_t) @ MatrixExp_4x4(xi_2_0, theta_2_t) @ T_c2_0_0

Joint positions and poses at $\vec{\theta}$.

In [12]:
# Positions
r_1 = Position(T_1_0)
r_2 = Position(T_2_0)

# Poses
omega_1 = MatrixExp_3x3(omega_1_0, theta_1_t) @ omega_1_0
omega_2 = (
    MatrixExp_3x3(omega_1_0, theta_1_t)
    @ MatrixExp_3x3(omega_2_0, theta_2_t)
    @ omega_2_0
)

Joint unitary twists at $\vec{\theta} $.

In [13]:
xi_1 = Joint2Twist(omega_1, r_1)
xi_2 = Joint2Twist(omega_2, r_2)

Link CoM postions.

In [14]:
r_c1 = Position(T_c1_0)
r_c2 = Position(T_c2_0)

Joints' spatial velocity.

In [15]:
# Rotation part of joints' Spatial Velocity
SpatialVelocity_Rot_1 = omega_1 * theta_1_t.diff(t)
SpatialVelocity_Rot_2 = omega_1 * \
    theta_1_t.diff(t) + omega_2 * theta_2_t.diff(t)

# Translation part of Joints' Spatial Velocity
SpatialVelocity_Trans_1 = r_1.diff(t)
SpatialVelocity_Trans_2 = r_2.diff(t)

# Joints' Spatial Velocity
SpatialVelocity_1 = SpatialVelocity_Rot_1.col_join(SpatialVelocity_Trans_1)
SpatialVelocity_2 = SpatialVelocity_Rot_2.col_join(SpatialVelocity_Trans_2)

CoMs' spatial velocity.

In [16]:
# Rotation part of CoMs' Spatial Velocity
SpatialVelocity_Rot_c1 = omega_1 * theta_1_t.diff(t)
SpatialVelocity_Rot_c2 = omega_1 * \
    theta_1_t.diff(t) + omega_2 * theta_2_t.diff(t)

# Translation part of CoMs' Spatial Velocity
SpatialVelocity_Trans_c1 = r_c1.diff(t)
SpatialVelocity_Trans_c2 = r_c2.diff(t)

# CoMs' Spatial Velocity
SpatialVelocity_c1 = SpatialVelocity_Rot_c1.col_join(SpatialVelocity_Trans_c1)
SpatialVelocity_c2 = SpatialVelocity_Rot_c2.col_join(SpatialVelocity_Trans_c2)

### 1.3 CoMs' Jacobians


CoMs' Jacobian matrix from definition.

In [17]:
def JacobianFromDefinition(V_CoM, Angles):
    # ! .coeff() may lose vital information, use .subs() instead
    # ? We compute Jacobian by substituting $\dot{\theta}_{i}$ with $1$,
    # ? the other with $0$ at column $i$.
    J_CoM = p.zeros(6, Angles.shape[0])
    for i in range(6):
        # i refers to row
        J_CoM[i, :] = V_CoM[i] * p.ones(1, Angles.shape[0])
        for j in range(Angles.shape[0]):
            # j refers to column
            for k in range(Angles.shape[0]):
                J_CoM[i, j] = J_CoM[i, j].subs(Angles[k].diff(t), int(j == k))
    return J_CoM


JointSpace = p.Matrix([theta_1_t, theta_2_t])

# Joint Jacobian
Jacobian_1 = p.simplify(ThetaT2Theta(
    JacobianFromDefinition(SpatialVelocity_1, JointSpace)))
Jacobian_2 = p.simplify(ThetaT2Theta(
    JacobianFromDefinition(SpatialVelocity_2, JointSpace)))

# Link CoM Jacobian
Jacobian_c1 = p.simplify(ThetaT2Theta(
    JacobianFromDefinition(SpatialVelocity_c1, JointSpace)))
Jacobian_c2 = p.simplify(ThetaT2Theta(
    JacobianFromDefinition(SpatialVelocity_c2, JointSpace)))

## 2. Dynamics

Define Generalized Inertial Matrixes, which follow

$$
M_{i} = \begin{bmatrix}
    I_{i} & \mathbf{0}_{3 \times 3} \\
    \mathbf{0}_{3 \times 3} & m_{i} \mathbf{I}_{3}
\end{bmatrix}.
$$

In [18]:
def MiMatrix(m, I):
    LowerRight = m * p.Identity(3).as_explicit()
    UpperRight, LowerLeft = p.zeros(3, 3), p.zeros(3, 3)
    return I.row_join(UpperRight).col_join(LowerLeft.row_join(LowerRight))

Compute $\mathbf{I}_{i}$ matrixes.

In [19]:
I1 = p.Matrix([[Jc1 / 2, 0, 0], [0, Jc1 / 2, 0], [0, 0, Jc1]])
I2 = p.Matrix([[Jc2 / 2, 0, 0], [0, Jc2 / 2, 0], [0, 0, Jc2]])

Compute Generalized Inertial Matrixes.

In [20]:
M1 = MiMatrix(m1, I1)
M2 = MiMatrix(m2, I2)

Compute Mass Matrix $M$

In [21]:
M = Jacobian_c1.T @ M1 @ Jacobian_c1 + Jacobian_c2.T @ M2 @ Jacobian_c2
M = p.simplify(M)
M

Matrix([
[Jc1 + Jc2 + l1**2*m2 + 2*l1*lc2*m2*cos(theta_2) + lc1**2*m1 + lc2**2*m2, Jc2 + l1*lc2*m2*cos(theta_2) + lc2**2*m2],
[                               Jc2 + l1*lc2*m2*cos(theta_2) + lc2**2*m2,                          Jc2 + lc2**2*m2]])

Compute Gravity Vector $\vec{G}$

In [22]:
PotentialEnergy = p.simplify(
    ThetaT2Theta(m1 * p.Matrix([[0, g, 0]]) @
                 r_c1 + m2 * p.Matrix([[0, g, 0]]) @ r_c2)
)
G = p.simplify(
    p.Matrix([[PotentialEnergy.diff(theta_1)],
             [PotentialEnergy.diff(theta_2)]])
)
G

Matrix([
[g*(l1*m2*cos(theta_1) + lc1*m1*cos(theta_1) + lc2*m2*cos(theta_1 + theta_2))],
[                                             g*lc2*m2*cos(theta_1 + theta_2)]])

Compute Coriolis Vector $\vec{h}$

In [23]:
def h_ijk(i, j, k):
    # ? i,j,k start from 1
    return (
        M[i - 1, j - 1].diff(ThetaT2Theta(JointSpace[k - 1]))
        + M[i - 1, k - 1].diff(ThetaT2Theta(JointSpace[j - 1]))
        - M[k - 1, j - 1].diff(ThetaT2Theta(JointSpace[i - 1]))
    ) / 2


h = p.zeros(2, 1)

dtheta_1, dtheta_2 = p.symbols("dtheta_1 dtheta_2")
dthetaSpace = p.Matrix([dtheta_1, dtheta_2])

for i in range(1, 3):
    # h vector's row
    for j in range(1, 3):
        for k in range(1, 3):
            h[i - 1, 0] += h_ijk(i, j, k) * \
                dthetaSpace[j - 1] * dthetaSpace[k - 1]

h

Matrix([
[-2*dtheta_1*dtheta_2*l1*lc2*m2*sin(theta_2) - dtheta_2**2*l1*lc2*m2*sin(theta_2)],
[                                              dtheta_1**2*l1*lc2*m2*sin(theta_2)]])

Export `Python` code.

In [24]:
p.printing.pycode(h)

'ImmutableDenseMatrix([[-2*dtheta_1*dtheta_2*l1*lc2*m2*math.sin(theta_2) - dtheta_2**2*l1*lc2*m2*math.sin(theta_2)], [dtheta_1**2*l1*lc2*m2*math.sin(theta_2)]])'

Compute $C$ matrix.


$$
\begin{aligned}
\sum_{j = 1}^{n} C_{ij} \dot{q}_{j} &= \sum_{j = 1}^{n} \sum_{k = 1}^{n} h_{ijk} \dot{q}_{j} \dot{q}_k \\
C_{ij} &= \sum_{k = 1}^{n} h_{ijk} \dot{q}_k,
\end{aligned}
$$

In [25]:
C = p.zeros(2, 2)

for i in range(1, 3):
    # C matrix's row
    for j in range(1, 3):
        for k in range(1, 3):
            C[i - 1, j - 1] += h_ijk(i, j, k) * dthetaSpace[k - 1]

C

Matrix([
[-dtheta_2*l1*lc2*m2*sin(theta_2), -dtheta_1*l1*lc2*m2*sin(theta_2) - dtheta_2*l1*lc2*m2*sin(theta_2)],
[ dtheta_1*l1*lc2*m2*sin(theta_2),                                                                  0]])

Verify

In [26]:
p.simplify(C @ p.Matrix([[dtheta_1], [dtheta_2]]) - h) == p.zeros(2, 1)

True

## 3. Model Linearization

Prettify

In [27]:
v1, v2 = p.symbols("v1 v2")
a1, a2 = p.symbols("a1 a2")

vSpace = p.Matrix([v1, v2])
aSpace = p.Matrix([a1, a2])

Define $\vec{p}$

In [28]:
ParaVec = p.Matrix(
    [
        Jc1 / 2 + m1 * (yc1**2 + zc1**2),
        0 + m1 * xc1 * yc1,
        0 + m1 * xc1 * zc1,
        Jc1 / 2 + m1 * (xc1**2 + zc1**2),
        0 + m1 * yc1 * zc1,
        Jc1 + m1 * (xc1**2 + yc1**2),
        m1 * xc1,
        m1 * yc1,
        m1 * zc1,
        m1,
    ]
).col_join(
    p.Matrix(
        [
            Jc2 / 2 + m2 * (yc2**2 + zc2**2),
            0 + m2 * xc2 * yc2,
            0 + m2 * xc2 * zc2,
            Jc2 / 2 + m2 * (xc2**2 + zc2**2),
            0 + m2 * yc2 * zc2,
            Jc2 + m2 * (xc2**2 + yc2**2),
            m2 * xc2,
            m2 * yc2,
            m2 * zc2,
            m2,
        ]
    )
)
ParaVec.T

Matrix([[Jc1/2, 0, 0, Jc1/2 + lc1**2*m1, 0, Jc1 + lc1**2*m1, lc1*m1, 0, 0, m1, Jc2/2, 0, 0, Jc2/2 + lc2**2*m2, 0, Jc2 + lc2**2*m2, lc2*m2, 0, 0, m2]])

Define a function to mark $0$ rows

In [29]:
def GetMarker(Vector):
    n1, n2 = Vector.shape
    Marker = p.ones(n1, 1)
    for i in range(n1):
        if Vector[i, :] == p.zeros(1, n2):
            Marker[i, :] = p.zeros(1, 1)
    return Marker

Mark $0$ elements in $\vec{p}$

In [30]:
Marker_ParaVec = GetMarker(ParaVec)

Define $K (\vec{\omega})$ function.

In [31]:
def KMatrix_3x6(Omega):
    omega_x, omega_y, omega_z = Omega[0, 0], Omega[1, 0], Omega[2, 0]
    return p.Matrix(
        [
            [omega_x, -omega_y, -omega_z, 0, 0, 0],
            [0, -omega_x, 0, omega_y, -omega_z, 0],
            [0, 0, -omega_x, 0, -omega_y, omega_z],
        ]
    )

Compute $\tilde{T}^{i}$

In [32]:
JointVelocityRotTuple = (SpatialVelocity_Rot_1, SpatialVelocity_Rot_2)
JointVelocityTransTuple = (SpatialVelocity_Trans_1, SpatialVelocity_Trans_2)
RotationMatrixCoMTuple = (T_c1_0[:3, :3], T_c2_0[:3, :3])

TtildeTuple = ()

for i in range(2):
    Ttilde_i_T_left = (
        JointVelocityRotTuple[i].T @ KMatrix_3x6(JointVelocityRotTuple[i])
    ) / 2
    Ttilde_i_T_mid = (
        JointVelocityTransTuple[i].T
        @ Vector2Matrix_3x3(JointVelocityRotTuple[i])
        @ RotationMatrixCoMTuple[i]
    )
    Ttilde_i_T_right = JointVelocityTransTuple[i].T @ JointVelocityTransTuple[i] / 2
    Ttilde_i = (Ttilde_i_T_left.row_join(
        Ttilde_i_T_mid).row_join(Ttilde_i_T_right)).T
    Ttilde_i = p.simplify(Ttilde_i)
    TtildeTuple = TtildeTuple + (Ttilde_i,)

Compute $\tilde{T}$

In [33]:
Ttilde = p.Matrix([[]])

for i in range(len(TtildeTuple)):
    Ttilde = Ttilde.row_join(TtildeTuple[i].T)

Ttilde

Matrix([[0, 0, 0, 0, 0, Derivative(theta_1(t), t)**2/2, 0, 0, 0, 0, 0, 0, 0, 0, 0, (Derivative(theta_1(t), t) + Derivative(theta_2(t), t))**2/2, l1*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*cos(theta_2(t))*Derivative(theta_1(t), t), -l1*(Derivative(theta_1(t), t) + Derivative(theta_2(t), t))*sin(theta_2(t))*Derivative(theta_1(t), t), 0, l1**2*Derivative(theta_1(t), t)**2/2]])

Mark 0 elements in $\tilde{T}$

In [34]:
Marker_Ttilde = GetMarker(Ttilde.T)

Compute $\tilde{V}^{i}$

In [35]:
VtildeTuple = ()
JointLocationTuple = (r_1, r_2)

for i in range(2):
    Vtilde_i_T_left = p.zeros(1, 6)
    Vtilde_i_T_mid = -p.Matrix([[0, -g, 0]]) @ RotationMatrixCoMTuple[i]
    Vtilde_i_T_right = -p.Matrix([[0, -g, 0]]) @ JointLocationTuple[i]
    Vtilde_i = (Vtilde_i_T_left.row_join(
        Vtilde_i_T_mid).row_join(Vtilde_i_T_right)).T
    Vtilde_i = p.simplify(Vtilde_i)
    VtildeTuple = VtildeTuple + (Vtilde_i,)

Compute $\tilde{V}$

In [36]:
Vtilde = p.Matrix([[]])

for i in range(len(VtildeTuple)):
    Vtilde = Vtilde.row_join(VtildeTuple[i].T)

Vtilde

Matrix([[0, 0, 0, 0, 0, 0, g*sin(theta_1(t)), g*cos(theta_1(t)), 0, 0, 0, 0, 0, 0, 0, 0, g*sin(theta_1(t) + theta_2(t)), g*cos(theta_1(t) + theta_2(t)), 0, g*l1*sin(theta_1(t))]])

Mark 0 elements in $\tilde{V}$

In [37]:
Marker_Vtilde = GetMarker(Vtilde.T)

Obtain final marker for simplification of $\tilde{T}$, $\tilde{V}$ and $\vec{p}$

In [38]:
FinalMarker = GetMarker(
    p.HadamardProduct(Marker_Ttilde + Marker_Vtilde,
                      Marker_ParaVec).as_explicit()
)
FinalMarker.T

Matrix([[0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]])

Define $0$ elements elimination function.

In [39]:
def EliminateZero_1(T_TILDE, V_TILDE, PARA_VEC, MARKER):
    Ttilde_NoZero, Vtilde_NoZero, ParaVec_NoZero = (
        p.Matrix([[]]),
        p.Matrix([[]]),
        p.Matrix([[]]),
    )
    for idx in range(MARKER.shape[0]):
        if MARKER[idx, 0]:
            Ttilde_NoZero = Ttilde_NoZero.row_join(T_TILDE[:, idx])
            Vtilde_NoZero = Vtilde_NoZero.row_join(V_TILDE[:, idx])
            ParaVec_NoZero = ParaVec_NoZero.col_join(PARA_VEC[idx, :])

    return Ttilde_NoZero, Vtilde_NoZero, ParaVec_NoZero

Eliminate $0$ elements

In [40]:
Ttilde, Vtilde, ParaVec = EliminateZero_1(Ttilde, Vtilde, ParaVec, FinalMarker)

Obtain $Y$ matrix

Define variable substitution function

In [41]:
def Subs(Variable):
    return (
        Variable.subs(theta_1_t.diff(t, 2), a1)
        .subs(theta_2_t.diff(t, 2), a2)
        .subs(theta_1_t.diff(t), v1)
        .subs(theta_2_t.diff(t), v2)
        .subs(theta_1_t, theta_1)
        .subs(theta_2_t, theta_2)
    )

Obtain $Y$ matrix

In [42]:
Y = p.simplify(
    Ttilde.jacobian(p.Matrix([theta_1_t.diff(t), theta_2_t.diff(t)])).diff(t)
    - Ttilde.jacobian(p.Matrix([theta_1_t, theta_2_t]))
    + Vtilde.jacobian(p.Matrix([theta_1_t, theta_2_t]))
).T
Y = Subs(Y)
Y

Matrix([
[a1, g*cos(theta_1), a1 + a2, 2*a1*l1*cos(theta_2) + a2*l1*cos(theta_2) + g*cos(theta_1 + theta_2) - 2*l1*v1*v2*sin(theta_2) - l1*v2**2*sin(theta_2), l1*(a1*l1 + g*cos(theta_1))],
[ 0,              0, a1 + a2,                                                  a1*l1*cos(theta_2) + g*cos(theta_1 + theta_2) + l1*v1**2*sin(theta_2),                           0]])

Parameter vector $\vec{p}$.

In [43]:
ParaVec.T

Matrix([[Jc1 + lc1**2*m1, lc1*m1, Jc2 + lc2**2*m2, lc2*m2, m2]])

Check $Y$ matrix correctness.

In [44]:
p.simplify(
    Y @ ParaVec
    - M @ p.Matrix([[a1, a2]]).T
    - h.subs(dtheta_1, v1).subs(dtheta_2, v2)
    - G
) == p.zeros(2, 1)

True